In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [118]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Example data for control group (df)
df = pd.DataFrame({
    'Age': [35, 45, 50, 60, 65],
    'Gender': [0, 1, 0, 1, 0],
    'BMI': [22, 25, 27, 30, 24],
    'Depression': [0, 1, 0, 1, 0],
    'Employment_status': [1, 0, 1, 0, 1],
    'Baseline_Physical_Activity': [10, 20, 15, 30, 25],
    'Followup_Physical_Activity': [15, 25, 20, 35, 30],
    'Baseline_Quality_of_life': [70, 60, 65, 75, 80],
    'Baseline_Pain': [3, 5, 2, 4, 3],
    'PA_change': [5, 5, 5, 5, 5]
})

# Example data for treatment group (dfg)
dfg = pd.DataFrame({
    'Age': [30, 40, 55, 65, 70],
    'Gender': [1, 0, 1, 0, 1],
    'BMI': [23, 26, 28, 31, 25],
    'Depression': [1, 0, 1, 0, 1],
    'Employment_status': [0, 1, 0, 1, 0],
    'Baseline_Physical_Activity': [12, 18, 14, 28, 26],
    'Followup_Physical_Activity': [17, 23, 19, 33, 29],
    'Baseline_Quality_of_life': [68, 62, 63, 78, 75],
    'Baseline_Pain': [4, 3, 3, 5, 2],
    'PA_change': [5, 5, 5, 5, 3]
})

# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
covariates = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
              'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)




/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3765512276.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3765512276.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)


In [119]:
# Print the control_data DataFrame with the weights column
control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment,weights
0,35,0,22,0,1,10,15,70,3,5,0,0.556596
1,45,1,25,1,0,20,25,60,5,5,0,0.611272
2,50,0,27,0,1,15,20,65,2,5,0,2.043404
3,60,1,30,1,0,30,35,75,4,5,0,1.159390
4,65,0,24,0,1,25,30,80,3,5,0,0.629338


In [120]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [121]:
dfg

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,30,1,23,1,0,12,17,68,4,5,1
1,40,0,26,0,1,18,23,62,3,5,1
2,55,1,28,1,0,14,19,63,3,5,1
3,65,0,31,0,1,28,33,78,5,5,1
4,70,1,25,1,0,26,29,75,2,3,1


In [122]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Example data for control group (df)
df = pd.DataFrame({
    'Age': [35, 45, 50, 60, 65],
    'Gender': [0, 1, 0, 1, 0],
    'BMI': [22, 25, 27, 30, 24],
    'Depression': [0, 1, 0, 1, 0],
    'Employment_status': [1, 0, 1, 0, 1],
    'Baseline_Physical_Activity': [10, 20, 15, 30, 25],
    'Followup_Physical_Activity': [15, 25, 20, 35, 30],
    'Baseline_Quality_of_life': [70, 60, 65, 75, 80],
    'Baseline_Pain': [3, 5, 2, 4, 3],
    'PA_change': [5, 5, 5, 5, 5]
})

# Example data for treatment group (dfg)
dfg = pd.DataFrame({
    'Age': [30, 40, 55, 65, 70],
    'Gender': [1, 0, 1, 0, 1],
    'BMI': [23, 26, 28, 31, 25],
    'Depression': [1, 0, 1, 0, 1],
    'Employment_status': [0, 1, 0, 1, 0],
    'Baseline_Physical_Activity': [12, 18, 14, 28, 26],
    'Followup_Physical_Activity': [17, 23, 19, 33, 29],
    'Baseline_Quality_of_life': [68, 62, 63, 78, 75],
    'Baseline_Pain': [4, 3, 3, 5, 2],
    'PA_change': [5, 5, 5, 5, 3]
})

# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
covariates = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
              'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Calculate the number of repetitions for each control data point
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
balanced_control_data = control_data.loc[control_data.index.repeat(control_data['reps'])].reset_index(drop=True)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the weighted means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means:\n", target_means)

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)

# Display the balanced control group dataframe
print("Balanced Control Group Data:")
print(balanced_control_data)

print("balanced_control_data.shape", balanced_control_data.shape)
print("control_data.shape", control_data.shape)


Balanced Control Group Means:
 Age                           52.0
Gender                         0.2
BMI                           26.0
Depression                     0.2
Employment_status              0.8
Baseline_Physical_Activity    19.0
Baseline_Quality_of_life      71.0
Baseline_Pain                  2.8
dtype: float64

Target Treatment Group Means:
 Age                           52.0
Gender                         0.6
BMI                           26.6
Depression                     0.6
Employment_status              0.4
Baseline_Physical_Activity    19.6
Baseline_Quality_of_life      69.2
Baseline_Pain                  3.4
dtype: float64
Balanced Control Group Data:
   Age  Gender  BMI  Depression  Employment_status  \
0   35       0   22           0                  1   
1   50       0   27           0                  1   
2   50       0   27           0                  1   
3   60       1   30           1                  0   
4   65       0   24           0                 

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3251530657.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3251530657.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3251530657.py:82: SettingWithCopy

In [123]:
print("balanced_control_data.shape",balanced_control_data.shape)
print("control_data.shape",df.shape)

balanced_control_data.shape (5, 11)
control_data.shape (5, 11)


In [124]:
print("balanced_control_data.shape",balanced_control_data.shape)
print("control_data.shape",df.shape)

balanced_control_data.shape (5, 11)
control_data.shape (5, 11)


In [125]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [126]:
balanced_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,50,0,27,0,1,15,20,65,2,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [127]:
control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment,weights,reps
0,35,0,22,0,1,10,15,70,3,5,0,0.556596,1
1,45,1,25,1,0,20,25,60,5,5,0,0.611272,0
2,50,0,27,0,1,15,20,65,2,5,0,2.043404,2
3,60,1,30,1,0,30,35,75,4,5,0,1.159390,1
4,65,0,24,0,1,25,30,80,3,5,0,0.629338,1


In [128]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Example data for control group (df)
df = pd.DataFrame({
    'Age': [35, 45, 50, 60, 65],
    'Gender': [0, 1, 0, 1, 0],
    'BMI': [22, 25, 27, 30, 24],
    'Depression': [0, 1, 0, 1, 0],
    'Employment_status': [1, 0, 1, 0, 1],
    'Baseline_Physical_Activity': [10, 20, 15, 30, 25],
    'Followup_Physical_Activity': [15, 25, 20, 35, 30],
    'Baseline_Quality_of_life': [70, 60, 65, 75, 80],
    'Baseline_Pain': [3, 5, 2, 4, 3],
    'PA_change': [5, 5, 5, 5, 5]
})

# Example data for treatment group (dfg)
dfg = pd.DataFrame({
    'Age': [30, 40, 55, 65, 70],
    'Gender': [1, 0, 1, 0, 1],
    'BMI': [23, 26, 28, 31, 25],
    'Depression': [1, 0, 1, 0, 1],
    'Employment_status': [0, 1, 0, 1, 0],
    'Baseline_Physical_Activity': [12, 18, 14, 28, 26],
    'Followup_Physical_Activity': [17, 23, 19, 33, 29],
    'Baseline_Quality_of_life': [68, 62, 63, 78, 75],
    'Baseline_Pain': [4, 3, 3, 5, 2],
    'PA_change': [5, 5, 5, 5, 3]
})

# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
# Include 'Baseline_Physical_Activity' for calculating weights
covariates_for_weighting = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
# Exclude 'Baseline_Physical_Activity' and 'PA_change' in the final balancing
covariates_for_balancing = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates_for_weighting].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates_for_weighting].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Calculate the number of repetitions for each control data point
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
balanced_control_data = control_data.loc[control_data.index.repeat(control_data['reps'])].reset_index(drop=True)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the weighted means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates_for_balancing].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means:\n", target_means[covariates_for_balancing])

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)

# Display the balanced control group dataframe
print("Balanced Control Group Data:")
print(balanced_control_data)

print("balanced_control_data.shape", balanced_control_data.shape)
print("control_data.shape", control_data.shape)


Balanced Control Group Means:
 Age                         48.0
Gender                       0.4
BMI                         26.2
Depression                   0.4
Employment_status            0.6
Baseline_Quality_of_life    67.0
Baseline_Pain                3.2
dtype: float64

Target Treatment Group Means:
 Age                         52.0
Gender                       0.6
BMI                         26.6
Depression                   0.6
Employment_status            0.4
Baseline_Quality_of_life    69.2
Baseline_Pain                3.4
dtype: float64
Balanced Control Group Data:
   Age  Gender  BMI  Depression  Employment_status  \
0   35       0   22           0                  1   
1   45       1   25           1                  0   
2   50       0   27           0                  1   
3   50       0   27           0                  1   
4   60       1   30           1                  0   

   Baseline_Physical_Activity  Followup_Physical_Activity  \
0                          10 

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/1313196985.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/1313196985.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/1313196985.py:86: SettingWithCopy

In [129]:
balanced_control_data


,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,50,0,27,0,1,15,20,65,2,5,0
4,60,1,30,1,0,30,35,75,4,5,0


In [130]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [131]:
dfg

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,30,1,23,1,0,12,17,68,4,5,1
1,40,0,26,0,1,18,23,62,3,5,1
2,55,1,28,1,0,14,19,63,3,5,1
3,65,0,31,0,1,28,33,78,5,5,1
4,70,1,25,1,0,26,29,75,2,3,1


In [132]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Example data for control group (df)
df = pd.DataFrame({
    'Age': [35, 45, 50, 60, 65],
    'Gender': [0, 1, 0, 1, 0],
    'BMI': [22, 25, 27, 30, 24],
    'Depression': [0, 1, 0, 1, 0],
    'Employment_status': [1, 0, 1, 0, 1],
    'Baseline_Physical_Activity': [10, 20, 15, 30, 25],
    'Followup_Physical_Activity': [15, 25, 20, 35, 30],
    'Baseline_Quality_of_life': [70, 60, 65, 75, 80],
    'Baseline_Pain': [3, 5, 2, 4, 3],
    'PA_change': [5, 5, 5, 5, 5]
})

# Example data for treatment group (dfg)
dfg = pd.DataFrame({
    'Age': [30, 40, 55, 65, 70],
    'Gender': [1, 0, 1, 0, 1],
    'BMI': [23, 26, 28, 31, 25],
    'Depression': [1, 0, 1, 0, 1],
    'Employment_status': [0, 1, 0, 1, 0],
    'Baseline_Physical_Activity': [12, 18, 14, 28, 26],
    'Followup_Physical_Activity': [17, 23, 19, 33, 29],
    'Baseline_Quality_of_life': [68, 62, 63, 78, 75],
    'Baseline_Pain': [4, 3, 3, 5, 2],
    'PA_change': [5, 5, 5, 5, 3]
})

# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
# Include 'Baseline_Physical_Activity' for calculating weights
covariates_for_weighting = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
# Exclude 'Baseline_Physical_Activity' and 'PA_change' in the final balancing
covariates_for_balancing = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates_for_weighting].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates_for_weighting].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Apply the weights to the control data covariates for balancing check
weighted_control_data = control_data[covariates_for_weighting].multiply(control_data['weights'], axis=0)

# Calculate the weighted means of the covariates in the control group
weighted_means = weighted_control_data.mean()
print("Weighted Control Group Means:\n", weighted_means)
print("\nTarget Treatment Group Means:\n", target_means)

# For the final balanced control group, we use the repetitions approach excluding Baseline_Physical_Activity and PA_change
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
balanced_control_data = control_data.loc[control_data.index.repeat(control_data['reps'])].reset_index(drop=True)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates_for_balancing].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means (for balancing):\n", target_means[covariates_for_balancing])

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)

# Display the balanced control group dataframe
print("Balanced Control Group Data:")
print(balanced_control_data)

print("balanced_control_data.shape", balanced_control_data.shape)
print("control_data.shape", df.shape)


Weighted Control Group Means:
 Age                           51.925737
Gender                         0.354132
BMI                           26.516926
Depression                     0.354132
Employment_status              0.645868
Baseline_Physical_Activity    19.791523
Baseline_Quality_of_life      69.152119
Baseline_Pain                  3.067706
dtype: float64

Target Treatment Group Means:
 Age                           52.0
Gender                         0.6
BMI                           26.6
Depression                     0.6
Employment_status              0.4
Baseline_Physical_Activity    19.6
Baseline_Quality_of_life      69.2
Baseline_Pain                  3.4
dtype: float64
Balanced Control Group Means:
 Age                         49.0
Gender                       0.2
BMI                         25.0
Depression                   0.2
Employment_status            0.8
Baseline_Quality_of_life    68.0
Baseline_Pain                3.0
dtype: float64

Target Treatment Group Means 

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2807959935.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2807959935.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2807959935.py:94: SettingWithCopy

In [133]:
weighted_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain
0,19.480845,0.000000,12.245102,0.000000,0.556596,5.565956,38.961689,1.669787
1,27.507243,0.611272,15.281802,0.611272,0.000000,12.225441,36.676324,3.056360
2,102.170195,0.000000,55.171906,0.000000,2.043404,30.651059,132.821254,4.086808
3,69.563398,1.159390,34.781699,1.159390,0.000000,34.781699,86.954248,4.637560
4,40.907002,0.000000,15.104124,0.000000,0.629338,15.733462,50.347080,1.888015


In [134]:
balanced_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,50,0,27,0,1,15,20,65,2,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [135]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [136]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Example data for control group (df)
df = pd.DataFrame({
    'Age': [35, 45, 50, 60, 65],
    'Gender': [0, 1, 0, 1, 0],
    'BMI': [22, 25, 27, 30, 24],
    'Depression': [0, 1, 0, 1, 0],
    'Employment_status': [1, 0, 1, 0, 1],
    'Baseline_Physical_Activity': [10, 20, 15, 30, 25],
    'Followup_Physical_Activity': [15, 25, 20, 35, 30],
    'Baseline_Quality_of_life': [70, 60, 65, 75, 80],
    'Baseline_Pain': [3, 5, 2, 4, 3],
    'PA_change': [5, 5, 5, 5, 5]
})

# Example data for treatment group (dfg)
dfg = pd.DataFrame({
    'Age': [30, 40, 55, 65, 70],
    'Gender': [1, 0, 1, 0, 1],
    'BMI': [23, 26, 28, 31, 25],
    'Depression': [1, 0, 1, 0, 1],
    'Employment_status': [0, 1, 0, 1, 0],
    'Baseline_Physical_Activity': [12, 18, 14, 28, 26],
    'Followup_Physical_Activity': [17, 23, 19, 33, 29],
    'Baseline_Quality_of_life': [68, 62, 63, 78, 75],
    'Baseline_Pain': [4, 3, 3, 5, 2],
    'PA_change': [5, 5, 5, 5, 3]
})

# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
# Include 'Baseline_Physical_Activity' for calculating weights
covariates_for_weighting = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
# Exclude 'Baseline_Physical_Activity' and 'PA_change' in the final balancing
covariates_for_balancing = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates_for_weighting].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates_for_weighting].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Apply the weights to the control data covariates for balancing check
weighted_control_data = control_data[covariates_for_weighting].multiply(control_data['weights'], axis=0)

# Calculate the weighted means of the covariates in the control group
weighted_means = weighted_control_data.mean()
print("Weighted Control Group Means:\n", weighted_means)
print("\nTarget Treatment Group Means:\n", target_means)

# For the final balanced control group, we use the repetitions approach excluding Baseline_Physical_Activity and PA_change
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
balanced_control_data = control_data.loc[control_data.index.repeat(control_data['reps'])].reset_index(drop=True)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates_for_balancing].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means (for balancing):\n", target_means[covariates_for_balancing])

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)



Weighted Control Group Means:
 Age                           51.925737
Gender                         0.354132
BMI                           26.516926
Depression                     0.354132
Employment_status              0.645868
Baseline_Physical_Activity    19.791523
Baseline_Quality_of_life      69.152119
Baseline_Pain                  3.067706
dtype: float64

Target Treatment Group Means:
 Age                           52.0
Gender                         0.6
BMI                           26.6
Depression                     0.6
Employment_status              0.4
Baseline_Physical_Activity    19.6
Baseline_Quality_of_life      69.2
Baseline_Pain                  3.4
dtype: float64
Balanced Control Group Means:
 Age                         52.0
Gender                       0.2
BMI                         26.0
Depression                   0.2
Employment_status            0.8
Baseline_Quality_of_life    71.0
Baseline_Pain                2.8
dtype: float64

Target Treatment Group Means 

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3887489463.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3887489463.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/3887489463.py:94: SettingWithCopy

In [137]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [138]:
# Display the balanced control group dataframe
print("Balanced Control Group Data:")
balanced_control_data



Balanced Control Group Data:


,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,50,0,27,0,1,15,20,65,2,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [139]:
weighted_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain
0,19.480845,0.000000,12.245102,0.000000,0.556596,5.565956,38.961689,1.669787
1,27.507243,0.611272,15.281802,0.611272,0.000000,12.225441,36.676324,3.056360
2,102.170195,0.000000,55.171906,0.000000,2.043404,30.651059,132.821254,4.086808
3,69.563398,1.159390,34.781699,1.159390,0.000000,34.781699,86.954248,4.637560
4,40.907002,0.000000,15.104124,0.000000,0.629338,15.733462,50.347080,1.888015


In [140]:
weighted_means

Age                           51.925737
Gender                         0.354132
BMI                           26.516926
Depression                     0.354132
Employment_status              0.645868
Baseline_Physical_Activity    19.791523
Baseline_Quality_of_life      69.152119
Baseline_Pain                  3.067706
dtype: float64

In [141]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Example data for control group (df)
df = pd.DataFrame({
    'Age': [35, 45, 50, 60, 65],
    'Gender': [0, 1, 0, 1, 0],
    'BMI': [22, 25, 27, 30, 24],
    'Depression': [0, 1, 0, 1, 0],
    'Employment_status': [1, 0, 1, 0, 1],
    'Baseline_Physical_Activity': [10, 20, 15, 30, 25],
    'Followup_Physical_Activity': [15, 25, 20, 35, 30],
    'Baseline_Quality_of_life': [70, 60, 65, 75, 80],
    'Baseline_Pain': [3, 5, 2, 4, 3],
    'PA_change': [5, 5, 5, 5, 5]
})

# Example data for treatment group (dfg)
dfg = pd.DataFrame({
    'Age': [30, 40, 55, 65, 70],
    'Gender': [1, 0, 1, 0, 1],
    'BMI': [23, 26, 28, 31, 25],
    'Depression': [1, 0, 1, 0, 1],
    'Employment_status': [0, 1, 0, 1, 0],
    'Baseline_Physical_Activity': [12, 18, 14, 28, 26],
    'Followup_Physical_Activity': [17, 23, 19, 33, 29],
    'Baseline_Quality_of_life': [68, 62, 63, 78, 75],
    'Baseline_Pain': [4, 3, 3, 5, 2],
    'PA_change': [5, 5, 5, 5, 3]
})

# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
# Include 'Baseline_Physical_Activity' for calculating weights
covariates_for_weighting = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
# Exclude 'Baseline_Physical_Activity' and 'PA_change' in the final balancing
covariates_for_balancing = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates_for_weighting].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates_for_weighting].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Apply the weights to the control data covariates for balancing check, excluding Baseline_Physical_Activity and PA_change
weighted_control_data = control_data[covariates_for_balancing].multiply(control_data['weights'], axis=0)
weighted_control_data['Baseline_Physical_Activity'] = control_data['Baseline_Physical_Activity']
weighted_control_data['PA_change'] = control_data['PA_change']

# Calculate the weighted means of the covariates in the control group
weighted_means = weighted_control_data[covariates_for_balancing].mean()
print("Weighted Control Group Means:\n", weighted_means)
print("\nTarget Treatment Group Means:\n", target_means[covariates_for_balancing])

# For the final balanced control group, we use the repetitions approach excluding Baseline_Physical_Activity and PA_change
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
repeated_rows = control_data.index.repeat(control_data['reps'])
balanced_control_data = control_data.loc[repeated_rows].reset_index(drop=True)

# Ensure Baseline_Physical_Activity remains unchanged
balanced_control_data['Baseline_Physical_Activity'] = np.repeat(control_data['Baseline_Physical_Activity'].values, control_data['reps'].values)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates_for_balancing].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means (for balancing):\n", target_means[covariates_for_balancing])

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)

# Display the balanced control group dataframe
print("Balanced Control Group Data:")
print(balanced_control_data)

print("balanced_control_data.shape", balanced_control_data.shape)
print("control_data.shape", df.shape)


/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/1108907595.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/1108907595.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/1108907595.py:96: SettingWithCopy

Weighted Control Group Means:
 Age                         51.925737
Gender                       0.354132
BMI                         26.516926
Depression                   0.354132
Employment_status            0.645868
Baseline_Quality_of_life    69.152119
Baseline_Pain                3.067706
dtype: float64

Target Treatment Group Means:
 Age                         52.0
Gender                       0.6
BMI                         26.6
Depression                   0.6
Employment_status            0.4
Baseline_Quality_of_life    69.2
Baseline_Pain                3.4
dtype: float64
Balanced Control Group Means:
 Age                         48.0
Gender                       0.4
BMI                         26.2
Depression                   0.4
Employment_status            0.6
Baseline_Quality_of_life    67.0
Baseline_Pain                3.2
dtype: float64

Target Treatment Group Means (for balancing):
 Age                         52.0
Gender                       0.6
BMI                

In [142]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,60,1,30,1,0,30,35,75,4,5,0
4,65,0,24,0,1,25,30,80,3,5,0


In [143]:
weighted_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Quality_of_life,Baseline_Pain,Baseline_Physical_Activity,PA_change
0,19.480845,0.000000,12.245102,0.000000,0.556596,38.961689,1.669787,10,5
1,27.507243,0.611272,15.281802,0.611272,0.000000,36.676324,3.056360,20,5
2,102.170195,0.000000,55.171906,0.000000,2.043404,132.821254,4.086808,15,5
3,69.563398,1.159390,34.781699,1.159390,0.000000,86.954248,4.637560,30,5
4,40.907002,0.000000,15.104124,0.000000,0.629338,50.347080,1.888015,25,5


In [144]:
balanced_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,35,0,22,0,1,10,15,70,3,5,0
1,45,1,25,1,0,20,25,60,5,5,0
2,50,0,27,0,1,15,20,65,2,5,0
3,50,0,27,0,1,15,20,65,2,5,0
4,60,1,30,1,0,30,35,75,4,5,0


In [158]:
# OAI dataset
df = pd.read_csv("preprocessed_OAI.csv")
# GLAD dataset
dfg = pd.read_csv("preprocessed_Glad.csv")
df.shape
print("OAI shape:", df.shape, "GLAD shape:", dfg.shape)

OAI shape: (4131, 11) GLAD shape: (7603, 11)


In [159]:
# Renaming columns in df for consistency
df.rename(columns={
    'V00AGE': 'Age',
    'P02SEX': 'Gender',
    'P01BMI': 'BMI',
    'V00CESD6': 'Depression',
    'V00CEMPLOY_employment': 'Employment_status',
    'V00KOOSQOL': 'Baseline_Quality_of_life',
    'V01KOOSQOL': 'Followup_Quality_of_life',
    'knee_pain_baseline': 'Baseline_Pain',
    'knee_pain_follow': 'Followup_Pain',
    'PA_baseline_category': 'Baseline_Physical_Activity',
    'PA_followup_category': 'Followup_Physical_Activity'
}, inplace=True)

dfg.rename(columns={
    'age': 'Age',
    'gender': 'Gender',
    'fysb_BMI': 'BMI',
    'ptb_14618_depression': 'Depression',
    'ptb_3777_employment': 'Employment_status',
    'ptb_koos_qol_score': 'Baseline_Quality_of_life',
    'pt12_koos_qol_score': 'Followup_Quality_of_life',
    'ptb_3764': 'Baseline_Pain',
    'pt12_3764': 'Followup_Pain',
    'ptb_4145': 'Baseline_Physical_Activity',
    'pt12_4145': 'Followup_Physical_Activity'
}, inplace=True)


In [160]:
# OAI dataset
##### Physical activity
# Ensure the columns are of type 'category'
df['Followup_Physical_Activity'] = df['Followup_Physical_Activity'].astype('category')
df['Baseline_Physical_Activity'] = df['Baseline_Physical_Activity'].astype('category')

# Convert categories to codes if they are ordinal
df['Followup_Physical_Activity'] = df['Followup_Physical_Activity'].cat.codes
df['Baseline_Physical_Activity'] = df['Baseline_Physical_Activity'].cat.codes

df['PA_change'] = df['Followup_Physical_Activity'] - df['Baseline_Physical_Activity']

##### Quality of life
df['QOL_change'] = df['Followup_Quality_of_life'] - df['Baseline_Quality_of_life']

##### Pain
df['pain_change'] = df['Followup_Pain'] - df['Baseline_Pain']

In [161]:
# GLAD Dataset
##### Physical activity
dfg['Baseline_Physical_Activity'] = dfg['Baseline_Physical_Activity'].astype('category')
dfg['Baseline_Physical_Activity'] = dfg['Baseline_Physical_Activity'].cat.codes

dfg['Followup_Physical_Activity'] = dfg['Followup_Physical_Activity'].astype('category')
dfg['Followup_Physical_Activity'] = dfg['Followup_Physical_Activity'].cat.codes

dfg['PA_change'] = dfg['Followup_Physical_Activity'] - dfg['Baseline_Physical_Activity']

##### Quality of life
dfg['QOL_change'] = dfg['Followup_Quality_of_life'] - dfg['Baseline_Quality_of_life']

##### Pain
dfg['pain_change'] = dfg['Followup_Pain'] - dfg['Baseline_Pain']

In [149]:
# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
# Include 'Baseline_Physical_Activity' for calculating weights
covariates_for_weighting = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
# Exclude 'Baseline_Physical_Activity' and 'PA_change' in the final balancing
covariates_for_balancing = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates_for_weighting].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates_for_weighting].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Apply the weights to the control data covariates for balancing check, excluding Baseline_Physical_Activity and PA_change
weighted_control_data = control_data[covariates_for_balancing].multiply(control_data['weights'], axis=0)
weighted_control_data['Baseline_Physical_Activity'] = control_data['Baseline_Physical_Activity']
weighted_control_data['PA_change'] = control_data['PA_change']

# Calculate the weighted means of the covariates in the control group
weighted_means = weighted_control_data[covariates_for_balancing].mean()
print("Weighted Control Group Means:\n", weighted_means)
print("\nTarget Treatment Group Means:\n", target_means[covariates_for_balancing])

# For the final balanced control group, we use the repetitions approach excluding Baseline_Physical_Activity and PA_change
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
repeated_rows = control_data.index.repeat(control_data['reps'])
balanced_control_data = control_data.loc[repeated_rows].reset_index(drop=True)

# Ensure Baseline_Physical_Activity remains unchanged
balanced_control_data['Baseline_Physical_Activity'] = np.repeat(control_data['Baseline_Physical_Activity'].values, control_data['reps'].values)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates_for_balancing].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means (for balancing):\n", target_means[covariates_for_balancing])

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)

# Display the balanced control group dataframe
print("Balanced Control Group Data:")
print(balanced_control_data)

print("balanced_control_data.shape", balanced_control_data.shape)
print("control_data.shape", df.shape)

Weighted Control Group Means:
 Age                         56.691017
Gender                       1.592205
BMI                         29.588865
Depression                   1.249382
Employment_status            1.285521
Baseline_Quality_of_life    45.058284
Baseline_Pain               47.220971
dtype: float64

Target Treatment Group Means:
 Age                         56.692227
Gender                       1.700250
BMI                         29.592791
Depression                   1.069578
Employment_status            1.156649
Baseline_Quality_of_life    45.055406
Baseline_Pain               47.219387
dtype: float64
Balanced Control Group Means:
 Age                         56.675623
Gender                       1.590656
BMI                         29.558848
Depression                   1.252481
Employment_status            1.288066
Baseline_Quality_of_life    44.707194
Baseline_Pain               47.526023
dtype: float64

Target Treatment Group Means (for balancing):
 Age            

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2031293188.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2031293188.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2031293188.py:64: SettingWithCopy

In [150]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Followup_Quality_of_life,Baseline_Pain,Followup_Pain,PA_change,QOL_change,pain_change,Treatment
0,78.0,1.0,27.6,1.0,3.0,0,0,50.00,68.75,80.0,50.0,0,18.75,-30.0,0
1,76.0,2.0,27.4,1.0,3.0,0,2,100.00,62.50,0.0,40.0,2,-37.50,40.0,0
2,69.0,1.0,29.8,1.0,3.0,0,3,81.25,100.00,10.0,0.0,3,18.75,-10.0,0
3,68.0,2.0,30.1,2.0,3.0,0,0,75.00,81.25,30.0,30.0,0,6.25,0.0,0
4,76.0,1.0,32.4,1.0,2.0,0,0,75.00,68.75,30.0,40.0,0,-6.25,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,9,8,100.00,100.00,0.0,0.0,-1,0.00,0.0,0
4127,59.0,2.0,30.5,1.0,1.0,9,5,43.75,68.75,70.0,70.0,-4,25.00,0.0,0
4128,70.0,1.0,33.9,1.0,3.0,9,4,100.00,100.00,0.0,0.0,-5,0.00,0.0,0
4129,52.0,1.0,30.7,1.0,1.0,9,7,31.25,37.50,50.0,70.0,-2,6.25,20.0,0


In [154]:
weighted_control_data.to_csv('testtttt-weighted_control_data.csv', index=False)

In [155]:
balanced_control_data.to_csv('testtttt-balanced_control_data.csv', index=False)

In [162]:
df = df.drop(columns=['QOL_change', 'pain_change', 'Followup_Pain', 'Followup_Quality_of_life', 'Followup_Physical_Activity'])

In [163]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change
0,78.0,1.0,27.6,1.0,3.0,0,50.00,80.0,0
1,76.0,2.0,27.4,1.0,3.0,0,100.00,0.0,2
2,69.0,1.0,29.8,1.0,3.0,0,81.25,10.0,3
3,68.0,2.0,30.1,2.0,3.0,0,75.00,30.0,0
4,76.0,1.0,32.4,1.0,2.0,0,75.00,30.0,0
...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,9,100.00,0.0,-1
4127,59.0,2.0,30.5,1.0,1.0,9,43.75,70.0,-4
4128,70.0,1.0,33.9,1.0,3.0,9,100.00,0.0,-5
4129,52.0,1.0,30.7,1.0,1.0,9,31.25,50.0,-2


In [164]:
# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
# Include 'Baseline_Physical_Activity' for calculating weights
covariates_for_weighting = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
# Exclude 'Baseline_Physical_Activity' and 'PA_change' in the final balancing
covariates_for_balancing = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates_for_weighting].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates_for_weighting].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Apply the weights to the control data covariates for balancing check, excluding Baseline_Physical_Activity and PA_change
weighted_control_data = control_data[covariates_for_balancing].multiply(control_data['weights'], axis=0)
weighted_control_data['Baseline_Physical_Activity'] = control_data['Baseline_Physical_Activity']
weighted_control_data['PA_change'] = control_data['PA_change']

# Calculate the weighted means of the covariates in the control group
weighted_means = weighted_control_data[covariates_for_balancing].mean()
print("Weighted Control Group Means:\n", weighted_means)
print("\nTarget Treatment Group Means:\n", target_means[covariates_for_balancing])

# For the final balanced control group, we use the repetitions approach excluding Baseline_Physical_Activity and PA_change
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
repeated_rows = control_data.index.repeat(control_data['reps'])
balanced_control_data = control_data.loc[repeated_rows].reset_index(drop=True)

# Ensure Baseline_Physical_Activity remains unchanged
balanced_control_data['Baseline_Physical_Activity'] = np.repeat(control_data['Baseline_Physical_Activity'].values, control_data['reps'].values)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates_for_balancing].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means (for balancing):\n", target_means[covariates_for_balancing])

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)

# Display the balanced control group dataframe
print("Balanced Control Group Data:")
print(balanced_control_data)

print("balanced_control_data.shape", balanced_control_data.shape)
print("control_data.shape", df.shape)

Weighted Control Group Means:
 Age                         56.691017
Gender                       1.592205
BMI                         29.588865
Depression                   1.249382
Employment_status            1.285521
Baseline_Quality_of_life    45.058284
Baseline_Pain               47.220971
dtype: float64

Target Treatment Group Means:
 Age                         56.692227
Gender                       1.700250
BMI                         29.592791
Depression                   1.069578
Employment_status            1.156649
Baseline_Quality_of_life    45.055406
Baseline_Pain               47.219387
dtype: float64
Balanced Control Group Means:
 Age                         56.679254
Gender                       1.591624
BMI                         29.595183
Depression                   1.252481
Employment_status            1.287582
Baseline_Quality_of_life    44.751069
Baseline_Pain               47.521181
dtype: float64

Target Treatment Group Means (for balancing):
 Age            

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2031293188.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2031293188.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_79659/2031293188.py:64: SettingWithCopy

In [168]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment
0,78.0,1.0,27.6,1.0,3.0,0,50.00,80.0,0,0
1,76.0,2.0,27.4,1.0,3.0,0,100.00,0.0,2,0
2,69.0,1.0,29.8,1.0,3.0,0,81.25,10.0,3,0
3,68.0,2.0,30.1,2.0,3.0,0,75.00,30.0,0,0
4,76.0,1.0,32.4,1.0,2.0,0,75.00,30.0,0,0
...,...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,9,100.00,0.0,-1,0
4127,59.0,2.0,30.5,1.0,1.0,9,43.75,70.0,-4,0
4128,70.0,1.0,33.9,1.0,3.0,9,100.00,0.0,-5,0
4129,52.0,1.0,30.7,1.0,1.0,9,31.25,50.0,-2,0


In [169]:
weighted_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Quality_of_life,Baseline_Pain,Baseline_Physical_Activity,PA_change
0,3.481549e-15,4.463525e-17,1.231933e-15,4.463525e-17,1.339057e-16,2.231762e-15,3.570820e-15,0,0
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,2
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,3
3,7.422446e+00,2.183072e-01,3.285524e+00,2.183072e-01,3.274608e-01,8.186521e+00,3.274608e+00,0,0
4,4.789204e+00,6.301585e-02,2.041713e+00,6.301585e-02,1.260317e-01,4.726189e+00,1.890475e+00,0,0
...,...,...,...,...,...,...,...,...,...
4126,4.833758e-15,8.631710e-17,2.080242e-15,8.631710e-17,8.631710e-17,8.631710e-15,0.000000e+00,9,-1
4127,1.361072e+02,4.613804e+00,7.036051e+01,2.306902e+00,2.306902e+00,1.009270e+02,1.614831e+02,9,-4
4128,1.254773e-17,1.792533e-19,6.076688e-18,1.792533e-19,5.377600e-19,1.792533e-17,0.000000e+00,9,-5
4129,2.065817e+02,3.972725e+00,1.219627e+02,3.972725e+00,3.972725e+00,1.241477e+02,1.986362e+02,9,-2


In [170]:
balanced_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change,Treatment,Followup_Physical_Activity,Followup_Quality_of_life,Followup_Pain,QOL_change,pain_change
0,70.0,1.0,28.9,1.0,1.0,0,81.25,30.0,0,0,NaN,NaN,NaN,NaN,NaN
1,77.0,1.0,33.6,1.0,3.0,0,37.50,50.0,3,0,NaN,NaN,NaN,NaN,NaN
2,49.0,1.0,23.5,1.0,1.0,0,81.25,20.0,6,0,NaN,NaN,NaN,NaN,NaN
3,67.0,2.0,34.7,1.0,3.0,0,62.50,0.0,3,0,NaN,NaN,NaN,NaN,NaN
4,56.0,2.0,23.9,1.0,1.0,0,100.00,0.0,5,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126,59.0,2.0,30.5,1.0,1.0,9,43.75,70.0,-4,0,NaN,NaN,NaN,NaN,NaN
4127,52.0,1.0,30.7,1.0,1.0,9,31.25,50.0,-2,0,NaN,NaN,NaN,NaN,NaN
4128,52.0,1.0,30.7,1.0,1.0,9,31.25,50.0,-2,0,NaN,NaN,NaN,NaN,NaN
4129,52.0,1.0,30.7,1.0,1.0,9,31.25,50.0,-2,0,NaN,NaN,NaN,NaN,NaN


In [173]:
balanced_control_data.to_csv('1.Balanced_control_data.csv', index=False)

In [174]:
weighted_control_data.to_csv('1.Weighted_control_data.csv', index=False)

In [178]:
mean_age_control = df['Age'].mean()
# Control group (with weights)
weighted_mean_age_control = weighted_control_data['Age'].mean()
# Treatment group (unweighted)
mean_age_treatment = dfg['Age'].mean()
# Balanced group
mean_age_balanced = balanced_control_data['Age'].mean()

print("Mean in control group:")
print(f"Mean Age (Control Group): {mean_age_control}")

print("--------------------------------------------------")
print("Mean in treatnement group:")
print(f"Mean Age (Treatment Group): {mean_age_treatment}")

print("--------------------------------------------------")
print("Weighted mean in control group:")
print(f"Weighted Mean Age (Control Group): {weighted_mean_age_control}")

print("--------------------------------------------------")
print("BALANCED mean in group:")
print(f"Balanced Mean Age (Control Group): {mean_age_balanced}")

Mean in control group:
Mean Age (Control Group): 61.08545146453643
--------------------------------------------------
Mean in treatnement group:
Mean Age (Treatment Group): 56.69222675259766
--------------------------------------------------
Weighted mean in control group:
Weighted Mean Age (Control Group): 56.69101749776496
--------------------------------------------------
BALANCED mean in group:
Balanced Mean Age (Control Group): 56.67925441781651
